<a href="https://colab.research.google.com/github/Vaycold/statistics_seminar/blob/main/covid19_tweet/simpletransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings 
import re

warnings.filterwarnings('ignore')

os.environ['KAGGLE_USERNAME'] = 'kimgodbang'
os.environ['KAGGLE_KEY'] = '1c0e1716b23829d4381dcbced37ba49b'

!rm *.*
!kaggle competitions download -c sentiment-analysis-of-covid-19-related-tweets

train_df = pd.read_csv('training.csv')
test_df = pd.read_csv('validation.csv')

  0% 0.00/518k [00:00<?, ?B/s]
100% 518k/518k [00:00<00:00, 76.5MB/s]
  0% 0.00/254k [00:00<?, ?B/s]
100% 254k/254k [00:00<00:00, 83.4MB/s]


In [2]:
train_df.head()

,ID,Tweet,Labels
0,1,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,0 10
1,2,BanMediaHouse whose is responsible for spreadi...,6
2,3,Im waiting for someone to say to me that all t...,3 4
3,4,He is a liar. Proven day night. Time again. Li...,6
4,5,"NEW: U.S. CoronaVirus death toll reaches 4,000...",8


In [3]:
def labelsplit(labels) :
    
    label = labels.replace(' ',',').split(',')
    for idx, lab in enumerate(label) :
        label[idx] = int(lab)
    return label

In [4]:
train_df['label']  = train_df['Labels'].apply(lambda x :  labelsplit(x))

In [5]:
import nltk

In [6]:
onehot = np.zeros((5000,11))
for i in range(5000) : # 0~5000 row
    for j in train_df.label[i] : # j = value   
        onehot[i,j]  = 1

onehot = pd.DataFrame(onehot, columns=['0','1','2','3','4','5','6','7','8','9','10'])
onehot[:3]

,0,1,2,3,4,5,6,7,8,9,10
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
train_df = pd.concat(
    [train_df, onehot], axis=1
).drop(['ID','Labels', 'label'],axis=1)

train_df.head()

,Tweet,0,1,2,3,4,5,6,7,8,9,10
0,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,BanMediaHouse whose is responsible for spreadi...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Im waiting for someone to say to me that all t...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,He is a liar. Proven day night. Time again. Li...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,"NEW: U.S. CoronaVirus death toll reaches 4,000...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [8]:
train_df.head()

,Tweet,0,1,2,3,4,5,6,7,8,9,10
0,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,BanMediaHouse whose is responsible for spreadi...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Im waiting for someone to say to me that all t...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,He is a liar. Proven day night. Time again. Li...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,"NEW: U.S. CoronaVirus death toll reaches 4,000...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [9]:
!pip install nltk

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
# Clean the textual column
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
def word_lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text

In [14]:
def  clean_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: word_tokenize(x))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: word_lemmatizer(x))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: ' '.join(x))
    return df

In [15]:
train_clean_df = clean_text(train_df, 'Tweet', 'clean_tweet')

In [16]:
train_clean_df

,Tweet,0,1,2,3,4,5,6,7,8,9,10,clean_tweet
0,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,joke hop plane rn well covid lol
1,BanMediaHouse whose is responsible for spreadi...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,banmediahouse whose responsible spreading fake...
2,Im waiting for someone to say to me that all t...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,im waiting someone say corona thing april fool...
3,He is a liar. Proven day night. Time again. Li...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,liar proven day night time lie truth covid
4,"NEW: U.S. CoronaVirus death toll reaches 4,000...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,new u coronavirus death toll reach nearly new ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,Life Insurance? I wonder if policies are payin...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,life insurance wonder policy paying corona dea...
4996,My cousin passed away from the corona virus to...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,cousin passed away corona virus today today da...
4997,I guess Mother Nature really hates us. Yellows...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,guess mother nature really hate u yellowstone ...
4998,So question for the day that isnt related to c...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,question day isnt related coronavirus better b...


In [17]:
train_clean_df.columns

Index(['Tweet', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       'clean_tweet'],
      dtype='object')

In [18]:
train = train_clean_df[['clean_tweet','0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']]
train.head()

,clean_tweet,0,1,2,3,4,5,6,7,8,9,10
0,joke hop plane rn well covid lol,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,banmediahouse whose responsible spreading fake...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,im waiting someone say corona thing april fool...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,liar proven day night time lie truth covid,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,new u coronavirus death toll reach nearly new ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [19]:
train['labels'] = train.apply(lambda x:
                              list([x['0'], x['1'], x['2'], x['3'], x['4'], x['5'], x['6'], x['7'], x['8'], x['9'],x['10']      ]), axis=1)
train.head()

,clean_tweet,0,1,2,3,4,5,6,7,8,9,10,labels
0,joke hop plane rn well covid lol,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,banmediahouse whose responsible spreading fake...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
2,im waiting someone say corona thing april fool...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
3,liar proven day night time lie truth covid,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
4,new u coronavirus death toll reach nearly new ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."


In [20]:
train_df = train[['clean_tweet','labels']]
train_df.columns = ['Tweet','labels']
train_df.head()

,Tweet,labels
0,joke hop plane rn well covid lol,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,banmediahouse whose responsible spreading fake...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
2,im waiting someone say corona thing april fool...,"[0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
3,liar proven day night time lie truth covid,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
4,new u coronavirus death toll reach nearly new ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."


In [21]:
!pip install simpletransformers

In [22]:
from simpletransformers.classification import MultiLabelClassificationModel

In [23]:
model = MultiLabelClassificationModel(
    'roberta',
    'roberta-base',
    num_labels = 11,
    args={
        'repreocess_input_data' : True,
        'num_train_epochs' : 5,
        'weight_decay' : 0.01,
        'evaluate_during_training_verbose' : True,

    }

)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight',

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [24]:
model.train_model(train_df)

  0%|          | 0/5000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/625 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/625 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/625 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/625 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/625 [00:00<?, ?it/s]

(3125, 0.3173498354959488)

In [25]:
test_df.head()

,ID,Tweet
0,5001,Forgot to a math test and I was failing but my...
1,5002,Corona effected came from Delhi in Bihar lakhi...
2,5003,Make CORONA END NA
3,5004,Imagine if the coronavirus pandemic was a big ...
4,5005,Howdy Ana. Where can I get married during the ...


In [26]:
test_clean_df = clean_text(test_df, 'Tweet', 'Clean_Tweet')
test_clean_df.head()

,ID,Tweet,Clean_Tweet
0,5001,Forgot to a math test and I was failing but my...,forgot math test failing teacher opening back ...
1,5002,Corona effected came from Delhi in Bihar lakhi...,corona effected came delhi bihar lakhisarai di...
2,5003,Make CORONA END NA,make corona end na
3,5004,Imagine if the coronavirus pandemic was a big ...,imagine coronavirus pandemic big april fool jo...
4,5005,Howdy Ana. Where can I get married during the ...,howdy ana get married coronavirus update gov


In [28]:
test_clean_df = test_clean_df[['Clean_Tweet']]
test_clean_df.head()

,Clean_Tweet
0,forgot math test failing teacher opening back ...
1,corona effected came delhi bihar lakhisarai di...
2,make corona end na
3,imagine coronavirus pandemic big april fool jo...
4,howdy ana get married coronavirus update gov


In [31]:
prediction_list = test_clean_df.Clean_Tweet.tolist()

In [32]:
predictions, raw_outputs = model.predict(prediction_list)

  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

In [33]:
test_clean_df['predictions'] = predictions

In [34]:
test_clean_df.head()

,Clean_Tweet,predictions
0,forgot math test failing teacher opening back ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
1,corona effected came delhi bihar lakhisarai di...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]"
2,make corona end na,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,imagine coronavirus pandemic big april fool jo...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
4,howdy ana get married coronavirus update gov,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"


In [37]:
answer = pd.DataFrame()
answer[['0','1','2','3','4','5','6','7','8','9','10']] = pd.DataFrame(
    test_clean_df['predictions'].tolist(), index = test_clean_df.index
)
answer

,0,1,2,3,4,5,6,7,8,9,10
0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1,0,0,1,0
2,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
2495,0,0,0,0,0,0,0,1,1,1,0
2496,0,0,0,1,0,1,1,0,0,0,0
2497,0,0,1,0,0,0,0,0,0,0,0
2498,0,0,0,0,0,0,0,0,0,0,1


In [39]:
cols_target = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

In [40]:
answer['answer'] = 0

In [42]:
for row in range(2500) :
    one_row = []
    for cat in cols_target :
        if int(answer[row:row+1][cat].values) == 1 :
            one_row.append(cat)
    answer.answer[row] = ' '.join(one_row)

In [44]:
Answer = answer[['answer']]
Answer.head()

,answer
0,10
1,6 9
2,0
3,10
4,10


In [47]:
submission = pd.concat(
    [test_df, Answer], axis=1
).drop(['Tweet','Clean_Tweet'], axis =1)
submission.head()

,ID,answer
0,5001,10
1,5002,6 9
2,5003,0
3,5004,10
4,5005,10


In [48]:
submission.columns = ['ID', 'Labels']
submission.head()

,ID,Labels
0,5001,10
1,5002,6 9
2,5003,0
3,5004,10
4,5005,10


In [49]:
submission.to_csv('submission.csv', index = False)